### Import required libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from CartPole import CartPole
from CartPole_GPS import CartPole_GPS

from ilqr.dynamics import constrain
from copy import deepcopy

from EstimateDynamics import local_estimate
from GMM import Estimated_Dynamics_Prior

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel

from mujoco_py import load_model_from_path, MjSim, MjViewer
import mujoco_py

import time

### Formulate the iLQR problem

In [5]:
'''
1 - dt = time step
2 - N = Number of control points in the trajectory
3 - x0 = Initial state
4 - x_goal = Final state
5 - Q = State cost
6 - R = Control cost
7 - Q_terminal = Cost at the final step
8 - x_dynamics array stores the information regarding system. 
    x_dynamics[0] = m = mass of the pendulum bob 
    x_dynamics[1] = M = mass of the cart 
    x_dynamics[2] = L = length of the massles|s rod 
    x_dynamics[3] = g = gravity 
    x_dynamics[4] = d = damping in the system
'''
dt = 0.005
N = 1500  # Number of time steps in trajectory.
x_dynamics = np.array([1, 5, 2, 9.80665, 0.1]) # m=1, M=5, L=0.3, g=9.80665, d=0.05
x0 = np.array([-2.0, 0.0, 0.01, 0.0])  # Initial state
x_goal = np.array([3.0, 0.0, 0.0, 0.0])
# Instantenous state cost.
Q = np.eye(5)
# Q[0,0] = 10
Q[2, 2] = 10
Q[3, 3] = 10
# Q[4, 4] = 100
# Terminal state cost.
Q_terminal = 100 * np.eye(5)
# Instantaneous control cost.
R = np.array([[1.0]])

### iLQR on Cart Pole

In [6]:
cartpole_prob = CartPole(dt, N, x_dynamics, x0, x_goal, Q, R, Q_terminal)
xs, us, K, k = cartpole_prob.run_IterLinQuadReg()

iteration 0 accepted 56628.75592722271 [ 4.9625241   0.84840578  1.88812918 -3.12305096]
iteration 1 accepted 56591.22358728466 [ 4.96629373  0.82611994  1.90615466 -3.1455061 ]
iteration 2 accepted 44929.60464025203 [ 2.68059412  0.82338615  1.51919855 -2.64581711]
iteration 3 accepted 44928.07137589979 [ 2.68781214  0.82987682  1.51291065 -2.63684383]
iteration 4 accepted 44927.40778580427 [ 2.6896289   0.83170082  1.51112155 -2.63429725]
iteration 5 failed 44927.40778580424 [ 2.6896289   0.83170082  1.51112155 -2.63429725]
iteration 6 failed 44927.40778580424 [ 2.6896289   0.83170082  1.51112155 -2.63429725]
iteration 7 failed 44927.40778580424 [ 2.6896289   0.83170082  1.51112155 -2.63429725]
iteration 8 failed 44927.40778580424 [ 2.6896289   0.83170082  1.51112155 -2.63429725]
iteration 9 failed 44927.40778580424 [ 2.6896289   0.83170082  1.51112155 -2.63429725]
iteration 10 failed 44927.40778580424 [ 2.6896289   0.83170082  1.51112155 -2.63429725]
iteration 11 accepted 44918.3491

iteration 93 failed 43572.98860813079 [ 2.99721824  0.25310184  2.17533542 -3.47350411]
iteration 94 accepted 43572.86857424622 [ 2.99728214  0.25203139  2.17604305 -3.47429273]
iteration 95 failed 43572.868574246226 [ 2.99728214  0.25203139  2.17604305 -3.47429273]
iteration 96 accepted 43572.4475066309 [ 2.99457594  0.25318964  2.17502328 -3.47318984]
iteration 97 failed 43572.44750663088 [ 2.99457594  0.25318964  2.17502328 -3.47318984]
iteration 98 accepted 43572.243627604126 [ 2.99448273  0.25273797  2.17524202 -3.4734287 ]
iteration 99 failed 43572.24362760409 [ 2.99448273  0.25273797  2.17524202 -3.4734287 ]


In [ ]:
# State matrix split into individual states. For plotting and analysing purposes.
t = np.arange(N + 1) * dt
x = xs[:, 0] # Position
x_dot = xs[:, 1] # Velocity
theta = np.unwrap(cartpole_prob.deaugment_state(xs)[:, 2])  # Theta, makes for smoother plots.
theta_dot = xs[:, 3] # Angular velocity

### Simulate the real system and generate the data
Cost matrices, initial position and goal position will remain same as the above problem. As it indicates one policy. But still the initial positions and goal positions must be passed explicitly to the function. But you don't need to pass cost matrices (assume penalty on the system is same), this is just used to use to calculate the cost of the trajectory. Correct control action must be passed. Parameter gamma indicates how much of original data you want to keep

Variance of the Gaussian noise will be taken as input from a Unif(0, var_range) uniform distribution. Inputs: x_initial, x_goal, u, n_rollouts, pattern='Normal', pattern_rand=False, var_range=10, gamma=0.2, percent=20

Pattern controls how the control sequence will be modified after applying white Guassian noise (zero mean).
- Normal: based on the correction/mixing parameter gamma generate control (gamma controls how much noise we want).
- MissingValue: based on the given percentage, set those many values to zero (it is implicitly it uses "Normal" generated control is used). 
- Shuffle: shuffles the entire "Normal" generated control sequence.
- TimeDelay: takes the "Normal" generated control and shifts it by 1 index i.e. one unit time delay.
- Extreme: sets gamma as zeros and generates control based on only noise.

If 'pattern_rand' is 'True' then we don't need to send the explicitly, it will chose one randomly for every rollout (default is 'False'). If you want to chose specific pattern then send it explicitly. 

In [ ]:
x_rollout, u_rollout, local_policy, cost = cartpole_prob.gen_rollouts(x0, x_goal, us, n_rollouts=10, pattern_rand=True, var_range=10, gamma=0.2, percent=20)

### Local system dynamics/model estimate
loca_estimate: function takes the states (arranged in a special format, [x(t), u(t), x(t+1)]), no. of gaussian mixtures and no.of states.

In [ ]:
model = Estimated_Dynamics_Prior(init_sequential=False, eigreg=False, warmstart=True, 
                 min_samples_per_cluster=20, max_clusters=50, max_samples=20, strength=1.0)
model.update_prior(x_rollout, u_rollout)
A, B, C = model.fit(x_rollout, u_rollout)

In [ ]:
print(A.shape)
print(B.shape)
print(C.shape)

In [ ]:
Model = "mujoco/cartpole.xml"
model_loaded = load_model_from_path(Model)
sim = MjSim(model_loaded)

In [ ]:
#viewer = mujoco_py.MjViewer(sim)
t = 0
sim.data.qpos[0] = 0.0
sim.data.qpos[1] = 0.01
sim.data.qvel[0] = 0
sim.data.qvel[1] = 0
final = 0
for i in range(1500):
    start_time = time.time()
    state = np.c_[sim.data.qpos[0],sim.data.qvel[0],np.sin(sim.data.qpos[1]),
                  np.cos(sim.data.qpos[1]),sim.data.qvel[1]].T
    control = np.dot(k[i,:],(state - xs[i].reshape(5,1) ))  + K[i].T + us[i]
    sim.data.ctrl[0] = control
    sim.step()
    #viewer.render()
    #print(control, "this is the control")
    if (sim.data.qpos[0] == 1.0 and sim.data.qpos[1] == 0):
        print('states reached')
        break
print(sim.get_state())

In [ ]:
from Simulator import Mujoco_sim
cart_pole_simulator = Mujoco_sim(Model,True)
cart_pole_simulator.load(xs,us,k,K,x0,initial=False)

In [ ]:
cart_pole_simulator.runSimulation()

In [ ]:
np.append(np.expand_dims(state,1),np.expand_dims(np.append(sim.data.qpos,sim.data.qvel),1),axis = 1)

In [ ]:
k.shape